In [1]:
# Install pip
# Note: Before run this coding, write pip install at the command centre of Anaconda Power Shell/terminal 
# pip3 install pandas dash
# pip install jupyter-dash

# Import required libraries
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update


# Create a dash application
app = JupyterDash(__name__)

### 1. Import required libraries

In [2]:
!pip3 install pandas dash
!pip3 install jupyter-dash

In [3]:
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update

In [4]:
import wget
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv'
filename = wget.download(url)
filename

100% [................................................................................] 2476 / 2476

'spacex_launch_dash (8).csv'

### 2. Read dataset

In [5]:
spacex_df = pd.read_csv("spacex_launch_dash (1).csv")

In [6]:
print ("Dataset records:")
print ("Rows     : " ,spacex_df.shape[0])
print ("Columns  : " ,spacex_df.shape[1])

Dataset records:
Rows     :  56
Columns  :  7


### 3. Create a dash application

In [7]:
app = JupyterDash(__name__)

In [8]:
app.config.suppress_callback_exceptions = True

In [10]:
JupyterDash.infer_jupyter_proxy_config()

### 4. Plotly Dash

In [ ]:
# Read the airline data into pandas dataframe
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Div(dcc.Dropdown(id='site-dropdown',
                                                      options=[{'label': 'All Sites', 'value': 'ALL'}, 
                                                               {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'}, 
                                                               {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}, 
                                                               {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'}, 
                                                               {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}],
                                                      value='ALL')),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                html.Div(dcc.RangeSlider(id='payload-slider',
                                                         min=0, max=10000, step=1000,
                                                        marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
                                                        value=[0,2500]
                                )),
                                html.Br(),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output

@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        filtered_df = spacex_df
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', 
            title='Total Success Launches for All Sites')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[['Launch Site','class']][spacex_df['Launch Site']==entered_site].groupby('class',as_index=False).count()
        filtered_df.rename(columns={"Launch Site": "count"},inplace=True)
        fig = px.pie(filtered_df, values='count', 
        names='class', 
            title='Total Success Launches for ' + entered_site)
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'), 
              Input(component_id="payload-slider", component_property="value"))

def get_scatter_chart(entered_site, entered_payload):
    if entered_site == 'ALL':
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)']>=entered_payload[0]) & (spacex_df['Payload Mass (kg)']<= entered_payload[1])]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
            title='Correlation between Payload and Success for All Sites')        
        return fig
    else:
        filtered_df = spacex_df[(spacex_df['Launch Site']==entered_site) & (spacex_df['Payload Mass (kg)']>=entered_payload[0]) & (spacex_df['Payload Mass (kg)']<= entered_payload[1])]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
            title='Correlation between Payload and Success for ' + entered_site)
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Jun/2022 00:27:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jun/2022 00:27:22] "GET /_dash-component-suites/dash/deps/react@16.v2_4_1m1652973876.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Jun/2022 00:27:22] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_4_1m1652973876.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Jun/2022 00:27:22] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_4_1m1652973875.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Jun/2022 00:27:22] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_4_1m1652973875.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Jun/2022 00:27:22] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_4_1m1652973874.min.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Jun/2022 00:27:22] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_4_0m1652973875.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Jun/2022 00:27:22] "GET /_dash-component-suites/d